In [1]:
import os
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import wandb

# 시드 고정
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# WANDB 초기화
wandb.login()
wandb.init(project='v6_x50_fold', name='polt-voting_eff-B7')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 경로 및 설정
data_path_train = 'data/augmented_v3_x50/'
data_path_test = 'data/test/'
img_size = 224
LR = 2e-5
EPOCHS = 50
BATCH_SIZE = 16
num_workers = 8
patience = 5
WEIGHT_DECAY = 1e-4
DROPOUT_PROB = 0.7

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df.values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = Image.open(os.path.join(self.path, name)).convert("RGB")
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# EfficientNet-B7 모델 정의
class CustomEfficientNetB7(nn.Module):
    def __init__(self, num_classes=17, dropout_prob=0.7):
        super(CustomEfficientNetB7, self).__init__()
        self.model = timm.create_model('tf_efficientnet_b7_ns', pretrained=True)   # tf_efficientnet_b7.ns_jft_in1k
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler, scheduler=None, clip_value=0.5):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, desc="Training")
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            preds = model(image)
            loss = loss_fn(preds, targets)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    if scheduler:
        scheduler.step()

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in tqdm(loader, desc="Validating"):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# augmentation을 위한 transform 정의
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터 로드
train_df_test = pd.read_csv('data/augment_v3_x50.csv')
test_df_test = pd.read_csv('data/sample_submission.csv')

# 클래스별 가중치 계산
class_counts = train_df_test['target'].value_counts().sort_index()
total_samples = len(train_df_test)
class_weights = [total_samples / class_counts[i] for i in range(len(class_counts))]
class_weights = np.array(class_weights)
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Stratified K-Fold 설정
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
folds = list(skf.split(train_df_test['ID'], train_df_test['target']))

# 학습 및 검증 루프
best_val_loss = float('inf')
fold_val_metrics = []
best_model_paths = []

for fold, (train_idx, val_idx) in enumerate(folds):
    if fold == 4:  # 5번째 폴드만 학습
        print(f"Fold {fold + 1}")

        trn_dataset = ImageDataset(
            train_df_test.iloc[train_idx],
            data_path_train,
            transform=trn_transform
        )

        val_dataset = ImageDataset(
            train_df_test.iloc[val_idx],
            data_path_train,
            transform=tst_transform
        )

        trn_loader = DataLoader(
            trn_dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True,
            drop_last=False
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )

        # 모델 정의 (EfficientNet-B7으로 변경)
        model = CustomEfficientNetB7(num_classes=17, dropout_prob=DROPOUT_PROB).to(device)

        loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)
        scaler = torch.cuda.amp.GradScaler()

        fold_best_val_loss = float('inf')
        fold_best_model_path = f'best_model_fold_{fold + 1}_fold_x50_Eff-B7.pth'
        patience_counter = 0

        for epoch in range(EPOCHS):
            train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler, scheduler)
            val_metrics = validate(val_loader, model, loss_fn, device)

            print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_metrics['train_loss']:.4f}, Train Acc: {train_metrics['train_acc']:.4f}, Train F1: {train_metrics['train_f1']:.4f}")
            print(f"Epoch {epoch + 1}/{EPOCHS}, Val Loss: {val_metrics['val_loss']:.4f}, Val Acc: {val_metrics['val_acc']:.4f}, Val F1: {val_metrics['val_f1']:.4f}")

            # WANDB 로그 기록
            wandb.log({
                'epoch': epoch + 1,
                'train_loss': train_metrics['train_loss'],
                'train_acc': train_metrics['train_acc'],
                'train_f1': train_metrics['train_f1'],
                'val_loss': val_metrics['val_loss'],
                'val_acc': val_metrics['val_acc'],
                'val_f1': val_metrics['val_f1']
            })

            scheduler.step(val_metrics['val_loss'])

            if val_metrics['val_loss'] < fold_best_val_loss:
                fold_best_val_loss = val_metrics['val_loss']
                torch.save(model.state_dict(), fold_best_model_path)
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break

        best_model_paths.append(fold_best_model_path)
        fold_val_metrics.append((fold_best_val_loss, val_metrics['val_f1']))
        print(f"Fold {fold + 1} Macro F1 Score: {val_metrics['val_f1']:.4f}")

wandb.finish()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kerynh (kerynhan). Use `wandb login --relogin` to force relogin


Fold 5


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(
Validating: 100%|██████████| 982/982 [01:18<00:00, 12.44it/s]


Epoch 1/50, Train Loss: 0.7895, Train Acc: 0.7542, Train F1: 0.7380
Epoch 1/50, Val Loss: 0.1047, Val Acc: 0.9646, Val F1: 0.9636


Validating: 100%|██████████| 982/982 [01:17<00:00, 12.72it/s]


...

Validating: 100%|██████████| 982/982 [02:32<00:00,  6.43it/s]


Epoch 8/50, Train Loss: 0.0372, Train Acc: 0.9911, Train F1: 0.9907
Epoch 8/50, Val Loss: 0.0119, Val Acc: 0.9980, Val F1: 0.9979


Validating: 100%|██████████| 982/982 [02:33<00:00,  6.39it/s]


Epoch 9/50, Train Loss: 0.0415, Train Acc: 0.9905, Train F1: 0.9900
Epoch 9/50, Val Loss: 0.0218, Val Acc: 0.9969, Val F1: 0.9966
Early stopping triggered at epoch 9
Fold 5 Macro F1 Score: 0.9966


epoch,▁▂▃▄▅▅▆▇█
train_acc,▁▇███████
train_f1,▁▇███████
train_loss,█▂▁▁▁▁▁▁▁
val_acc,▁████████
val_f1,▁████████
val_loss,█▁▁▁▂▁▁▁▂
epoch,9
train_acc,0.99049
train_f1,0.98996
train_loss,0.04153


In [ ]:
# import os
# import random
# import timm
# import torch
# import albumentations as A
# import pandas as pd
# import numpy as np
# import torch.nn as nn
# from albumentations.pytorch import ToTensorV2
# from torch.optim import Adam
# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import StratifiedKFold
# from PIL import Image
# import wandb

# # 시드 고정
# SEED = 42
# os.environ['PYTHONHASHSEED'] = str(SEED)
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.cuda.manual_seed_all(SEED)
# torch.backends.cudnn.benchmark = True

# # WANDB 초기화
# wandb.login()
# wandb.init(project='v6_x50_fold', name='polt-voting_eff-B7')

# # 디바이스 설정
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # 데이터 경로 및 설정
# data_path_train = 'data/augmented_v3_x50/'
# data_path_test = 'data/test/'
# img_size = 224
# LR = 2e-5
# EPOCHS = 50
# BATCH_SIZE = 16
# num_workers = 8
# patience = 5
# WEIGHT_DECAY = 1e-4
# DROPOUT_PROB = 0.7

# # 데이터셋 클래스 정의
# class ImageDataset(Dataset):
#     def __init__(self, df, path, transform=None):
#         self.df = df.values
#         self.path = path
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         name, target = self.df[idx]
#         img = Image.open(os.path.join(self.path, name)).convert("RGB")
#         img = np.array(img)
#         if self.transform:
#             img = self.transform(image=img)['image']
#         return img, target

# # EfficientNet-B7 모델 정의
# class CustomEfficientNetB7(nn.Module):
#     def __init__(self, num_classes=17, dropout_prob=0.7):
#         super(CustomEfficientNetB7, self).__init__()
#         self.model = timm.create_model('tf_efficientnet_b7_ns', pretrained=True)   # tf_efficientnet_b7.ns_jft_in1k
#         num_features = self.model.classifier.in_features
#         self.model.classifier = nn.Sequential(
#             nn.BatchNorm1d(num_features),
#             nn.Linear(num_features, 1024),
#             nn.BatchNorm1d(1024),
#             nn.ReLU(),
#             nn.Dropout(dropout_prob),
#             nn.Linear(1024, num_classes)
#         )

#     def forward(self, x):
#         return self.model(x)

# # 학습 함수 정의
# def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler, scheduler=None, clip_value=0.5):
#     model.train()
#     train_loss = 0
#     preds_list = []
#     targets_list = []

#     pbar = tqdm(loader, desc="Training")
#     for image, targets in pbar:
#         image = image.to(device)
#         targets = targets.to(device)

#         optimizer.zero_grad()
#         with torch.cuda.amp.autocast():
#             preds = model(image)
#             loss = loss_fn(preds, targets)
        
#         scaler.scale(loss).backward()
#         scaler.unscale_(optimizer)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
#         scaler.step(optimizer)
#         scaler.update()

#         train_loss += loss.item()
#         preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
#         targets_list.extend(targets.detach().cpu().numpy())

#         pbar.set_description(f"Loss: {loss.item():.4f}")

#     if scheduler:
#         scheduler.step()

#     train_loss /= len(loader)
#     train_acc = accuracy_score(targets_list, preds_list)
#     train_f1 = f1_score(targets_list, preds_list, average='macro')

#     ret = {
#         "train_loss": train_loss,
#         "train_acc": train_acc,
#         "train_f1": train_f1,
#     }

#     return ret

# # 검증 함수 정의
# def validate(loader, model, loss_fn, device):
#     model.eval()
#     val_loss = 0
#     preds_list = []
#     targets_list = []

#     with torch.no_grad():
#         for image, targets in tqdm(loader, desc="Validating"):
#             image = image.to(device)
#             targets = targets.to(device)

#             preds = model(image)
#             loss = loss_fn(preds, targets)

#             val_loss += loss.item()
#             preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
#             targets_list.extend(targets.detach().cpu().numpy())

#     val_loss /= len(loader)
#     val_acc = accuracy_score(targets_list, preds_list)
#     val_f1 = f1_score(targets_list, preds_list, average='macro')

#     ret = {
#         "val_loss": val_loss,
#         "val_acc": val_acc,
#         "val_f1": val_f1,
#     }

#     return ret

# # augmentation을 위한 transform 정의
# trn_transform = A.Compose([
#     A.Resize(height=img_size, width=img_size),
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(p=0.2),
#     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
#     A.CoarseDropout(p=0.5),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

# tst_transform = A.Compose([
#     A.Resize(height=img_size, width=img_size),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

# # 데이터 로드
# train_df_test = pd.read_csv('data/augment_v3_x50.csv')
# test_df_test = pd.read_csv('data/sample_submission.csv')

# # 클래스별 가중치 계산
# class_counts = train_df_test['target'].value_counts().sort_index()
# total_samples = len(train_df_test)
# class_weights = [total_samples / class_counts[i] for i in range(len(class_counts))]
# class_weights = np.array(class_weights)
# class_weights = class_weights / class_weights.sum() * len(class_counts)
# class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# # Stratified K-Fold 설정
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
# folds = list(skf.split(train_df_test['ID'], train_df_test['target']))

# # 학습 및 검증 루프
# best_val_loss = float('inf')
# fold_val_metrics = []
# best_model_paths = []

# for fold, (train_idx, val_idx) in enumerate(folds):
#     print(f"Fold {fold + 1}")

#     trn_dataset = ImageDataset(
#         train_df_test.iloc[train_idx],
#         data_path_train,
#         transform=trn_transform
#     )

#     val_dataset = ImageDataset(
#         train_df_test.iloc[val_idx],
#         data_path_train,
#         transform=tst_transform
#     )

#     trn_loader = DataLoader(
#         trn_dataset,
#         batch_size=BATCH_SIZE,
#         shuffle=True,
#         num_workers=num_workers,
#         pin_memory=True,
#         drop_last=False
#     )

#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=BATCH_SIZE,
#         shuffle=False,
#         num_workers=num_workers,
#         pin_memory=True
#     )

#     # 모델 정의 (EfficientNet-B7으로 변경)
#     model = CustomEfficientNetB7(num_classes=17, dropout_prob=DROPOUT_PROB).to(device)

#     loss_fn = nn.CrossEntropyLoss(weight=class_weights)
#     optimizer = Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
#     scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)
#     scaler = torch.cuda.amp.GradScaler()

#     fold_best_val_loss = float('inf')
#     fold_best_model_path = f'best_model_fold_{fold + 1}_fold_x50_Eff-B7.pth'
#     patience_counter = 0

#     for epoch in range(EPOCHS):
#         train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler, scheduler)
#         val_metrics = validate(val_loader, model, loss_fn, device)

#         print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_metrics['train_loss']:.4f}, Train Acc: {train_metrics['train_acc']:.4f}, Train F1: {train_metrics['train_f1']:.4f}")
#         print(f"Epoch {epoch + 1}/{EPOCHS}, Val Loss: {val_metrics['val_loss']:.4f}, Val Acc: {val_metrics['val_acc']:.4f}, Val F1: {val_metrics['val_f1']:.4f}")

#         # WANDB 로그 기록
#         wandb.log({
#             'epoch': epoch + 1,
#             'train_loss': train_metrics['train_loss'],
#             'train_acc': train_metrics['train_acc'],
#             'train_f1': train_metrics['train_f1'],
#             'val_loss': val_metrics['val_loss'],
#             'val_acc': val_metrics['val_acc'],
#             'val_f1': val_metrics['val_f1']
#         })

#         scheduler.step(val_metrics['val_loss'])

#         if val_metrics['val_loss'] < fold_best_val_loss:
#             fold_best_val_loss = val_metrics['val_loss']
#             torch.save(model.state_dict(), fold_best_model_path)
#             patience_counter = 0
#         else:
#             patience_counter += 1

#         if patience_counter >= patience:
#             print(f"Early stopping triggered at epoch {epoch + 1}")
#             break

#     best_model_paths.append(fold_best_model_path)
#     fold_val_metrics.append((fold_best_val_loss, val_metrics['val_f1']))
#     print(f"Fold {fold + 1} Macro F1 Score: {val_metrics['val_f1']:.4f}")

# wandb.finish()



In [2]:
# 테스트 데이터 예측 및 결과 저장
test_dataset = ImageDataset(test_df_test, data_path_test, transform=tst_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

model_paths = [
    'best_model_fold_1_fold_x50_Eff-B7.pth',
    'best_model_fold_2_fold_x50_Eff-B7.pth',
    'best_model_fold_3_fold_x50_Eff-B7.pth',
    'best_model_fold_4_fold_x50_Eff-B7.pth',
    'best_model_fold_5_fold_x50_Eff-B7.pth'
]

fold_preds = []
val_f1_scores = []
val_acc_scores = []
val_losses = []

for fold, (train_idx, val_idx) in enumerate(folds):
    model = CustomEfficientNetB7(num_classes=17, dropout_prob=0.7).to(device)
    model.load_state_dict(torch.load(model_paths[fold]))
    model.eval()

    val_dataset = ImageDataset(train_df_test.iloc[val_idx], data_path_train, transform=tst_transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    val_preds = []
    val_targets = []
    val_loss = 0.0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            output = model(images)
            loss = loss_fn(output, targets)
            val_loss += loss.item()

            val_preds.extend(output.argmax(dim=1).cpu().numpy())
            val_targets.extend(targets.cpu().numpy())

    val_loss /= len(val_loader)
    val_f1 = f1_score(val_targets, val_preds, average='macro')
    val_acc = accuracy_score(val_targets, val_preds)

    val_f1_scores.append(val_f1)
    val_acc_scores.append(val_acc)
    val_losses.append(val_loss)

    print(f"Fold {fold + 1} Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}, Validation F1 Score: {val_f1:.4f}")

    preds = []
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            output = model(images)
            preds.append(output.softmax(dim=1).cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    fold_preds.append(preds)

# 폴드별 예측을 평균하여 최종 예측 생성
final_preds = np.mean(fold_preds, axis=0)
final_class_preds = np.argmax(final_preds, axis=1)

# 결과 저장
submission = pd.DataFrame({'ID': test_df_test['ID'], 'target': final_class_preds})
submission.to_csv('submit_v6_x50_fold_Eff-B8.csv', index=False)

# 최종 검증 성능 출력
print(f"Mean Validation Loss: {np.mean(val_losses):.4f}, Mean Validation Accuracy: {np.mean(val_acc_scores):.4f}, Mean Validation F1 Score: {np.mean(val_f1_scores):.4f}")
print("최종 제출 파일이 저장되었습니다.")


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


Fold 1 Validation Loss: 0.0036, Validation Accuracy: 0.9992, Validation F1 Score: 0.9993


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


Fold 2 Validation Loss: 0.0049, Validation Accuracy: 0.9987, Validation F1 Score: 0.9987


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


Fold 3 Validation Loss: 0.0056, Validation Accuracy: 0.9989, Validation F1 Score: 0.9989


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


Fold 4 Validation Loss: 0.0046, Validation Accuracy: 0.9988, Validation F1 Score: 0.9988


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b7_ns to current tf_efficientnet_b7.ns_jft_in1k.
  model = create_fn(


Fold 5 Validation Loss: 0.0051, Validation Accuracy: 0.9989, Validation F1 Score: 0.9989
Mean Validation Loss: 0.0047, Mean Validation Accuracy: 0.9989, Mean Validation F1 Score: 0.9989
최종 제출 파일이 저장되었습니다.
